## Preparing Data

As in the previous notebooks, we'll prepare the data. 

Unlike the previous notebook with the FastText model, we no longer explicitly need to create the bi-grams and append them to the end of the sentence.

As convolutional layers expect the batch dimension to be first we can tell TorchText to return the data already permuted using the `batch_first = True` argument on the field.

In [0]:
import torch
from torchtext import data
from torchtext import datasets
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', batch_first = True)
LABEL = data.LabelField(dtype = torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

Build the vocab and load the pre-trained word embeddings.

In [0]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

As before, we create the iterators.

In [0]:
# print(', '.join(i for i in dir(TEXT.unk_token) if not i.startswith('__')))
# print(TEXT.unk_token.title)
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [0]:
import torch.nn as nn
import torch.nn.functional as F



We can also implement the above model using 1-dimensional convolutional layers, where the embedding dimension is the "depth" of the filter and the number of tokens in the sentence is the width.

We'll run our tests in this notebook using the 2-dimensional convolutional model, but leave the implementation for the 1-dimensional model below for anyone interested. 

In [0]:
class CNN1d(nn.Module):
    def __init__(self, vocab_size, embedding_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv1d(in_channels = embedding_dim, 
                                              out_channels = 50, 
                                              kernel_size = fs)
                                    for fs in [4,5]
                                    ])
        self.convs2 = nn.Conv1d(in_channels = 50, out_channels = 100, kernel_size = 3)
                                    
        self.fc1 = nn.Linear(149700, 100) #fix this
        self.fc2=nn.Linear(100,1)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
        
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.permute(0, 2, 1)
        #print("embedded",embedded.size())
        x=embedded.size(2)
        y=3000-x
        print(y)
        tsiz=embedded.size(0)
        z=np.zeros((tsiz,100,y))
        z1=torch.from_numpy(z).float()
        lz=[embedded,z1]
        #print(type(lz))
        zcat = torch.cat(lz, dim = 2)
        #print("zcat",zcat.size())
        #embedded = [batch size, emb dim, sent len]
        # print("Yo")
        conved = [F.relu(conv(zcat)) for conv in self.convs]
        # print("conved")
        # for c in conved:
        #     print(c.size())
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled = [F.max_pool1d(conv, 2) for conv in conved]
        # print("Pool:")
        # for pl in pooled:
        #   print(pl.size())
        cat = self.dropout(torch.cat(pooled, dim = 2))
        #pooled_n = [batch size, n_filters]
        #print("cat",cat.size())
        conved2 = F.relu(self.convs2(cat)) 
        #print("conved2",conved2.size())
        pooled2 = F.max_pool1d(conved2, 2).reshape(tsiz,149700)
        #print("pooled2",pooled2.size())
        
        #cat = [batch size, n_filters * len(filter_sizes)]
        # print("Gonna get error now")
        # print(pooled2.size())
        full1= self.fc1(pooled2)
        full2= self.fc2(full1)
        #print("done!")
        return full2

We create an instance of our `CNN` class. 

We can change `CNN` to `CNN1d` if we want to run the 1-dimensional convolutional model, noting that both models give almost identical results.

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN1d(INPUT_DIM, EMBEDDING_DIM, DROPOUT, PAD_IDX)

Checking the number of parameters in our model we can see it has about the same as the FastText model. 

Both the `CNN` and the `CNN1d` models have the exact same number of parameters.

In [148]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 17,530,601 trainable parameters


Next, we'll load the pre-trained embeddings

In [149]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-1.1172e-01, -4.9659e-01,  1.6307e-01,  ...,  1.2647e+00,
         -2.7527e-01, -1.3254e-01],
        [-8.5549e-01, -7.2081e-01,  1.3755e+00,  ...,  8.2522e-02,
         -1.1314e+00,  3.9972e-01],
        [-3.8194e-02, -2.4487e-01,  7.2812e-01,  ..., -1.4590e-01,
          8.2780e-01,  2.7062e-01],
        ...,
        [ 2.0009e-01,  3.2409e-01, -2.3066e-01,  ..., -7.4463e-02,
         -5.0526e-01,  7.2131e-04],
        [ 7.5567e-01,  4.1539e-01,  3.5299e-01,  ..., -3.6261e-01,
         -9.9088e-01,  1.0656e+00],
        [-8.3036e-01,  3.7317e-01,  7.2612e-02,  ..., -1.2178e-02,
          2.3129e-01, -2.7827e-01]])

Then zero the initial weights of the unknown and padding tokens.

In [0]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

## Train the Model

Training is the same as before. We initialize the optimizer, loss function (criterion) and place the model and criterion on the GPU (if available)

In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

We implement the function to calculate accuracy...

In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

We define a function for training our model...

**Note**: as we are using dropout again, we must remember to use `model.train()` to ensure the dropout is "turned on" while training.

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        #print("Error here",model(batch.text),model(batch.text).size())
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

We define a function for testing our model...

**Note**: again, as we are now using dropout, we must remember to use `model.eval()` to ensure the dropout is "turned off" while evaluating.

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Let's define our function to tell us how long epochs take.

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Finally, we train our model...

In [156]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Streaming output truncated to the last 5000 lines.
torch.Size([64, 149700])
2856
Gonna get error now
torch.Size([64, 149700])
2855
Gonna get error now
torch.Size([64, 149700])
2854
Gonna get error now
torch.Size([64, 149700])
2853
Gonna get error now
torch.Size([64, 149700])
2851
Gonna get error now
torch.Size([64, 149700])
2850
Gonna get error now
torch.Size([64, 149700])
2849
Gonna get error now
torch.Size([64, 149700])
2847
Gonna get error now
torch.Size([64, 149700])
2846
Gonna get error now
torch.Size([64, 149700])
2844
Gonna get error now
torch.Size([64, 149700])
2843
Gonna get error now
torch.Size([64, 149700])
2842
Gonna get error now
torch.Size([64, 149700])
2840
Gonna get error now
torch.Size([64, 149700])
2839
Gonna get error now
torch.Size([64, 149700])
2837
Gonna get error now
torch.Size([64, 149700])
2835
Gonna get error now
torch.Size([64, 149700])
2834
Gonna get error now
torch.Size([64, 149700])
2832
Gonna get error now
torch.Size([64, 149700])
2830
Gonna get error now

We get test results comparable to the previous 2 models!

## User Input

And again, as a sanity check we can check some input sentences

**Note**: As mentioned in the implementation details, the input sentence has to be at least as long as the largest filter height used. We modify our `predict_sentiment` function to also accept a minimum length argument. If the tokenized input sentence is less than `min_len` tokens, we append padding tokens (`<pad>`) to make it `min_len` tokens.

In [0]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence, min_len = 5):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

An example negative review...

In [171]:
predict_sentiment(model, "This is okay")

2992
Gonna get error now
torch.Size([1, 149700])


0.5960265398025513

An example positive review...

In [169]:
predict_sentiment(model, "This is the best movie I've ever watched!")

2990
Gonna get error now
torch.Size([1, 149700])


0.7516406178474426